In [2]:
import polars as pl
import cupy as cp

In [35]:
df = pl.read_parquet('flights2024.parquet')
df.head()

ITIN_ID,MKT_ID,MARKET_COUPONS,YEAR,QUARTER,ORIGIN,ORIGIN_WAC,DEST,DEST_WAC,WAC_GROUP,TK_CARRIER_CHANGE,OP_CARRIER_CHANGE,REPORTING_CARRIER,BULK_FARE,PASSENGERS,MARKET_FARE,MARKET_DISTANCE,MARKET_MILES_FLOWN
i64,i64,i64,i64,i64,str,i64,str,i64,str,f64,f64,str,f64,f64,f64,f64,f64
20241514,2024151401,1,2024,1,"""EYW""",33,"""FLL""",33,"""33:33""",0.0,0.0,"""3M""",0.0,1.0,0.0,145.0,145.0
20241515,2024151501,1,2024,1,"""EYW""",33,"""FLL""",33,"""33:33""",0.0,0.0,"""3M""",0.0,1.0,69.0,145.0,145.0
20241516,2024151601,1,2024,1,"""EYW""",33,"""FLL""",33,"""33:33""",0.0,0.0,"""3M""",0.0,2.0,77.0,145.0,145.0
20241517,2024151701,1,2024,1,"""EYW""",33,"""FLL""",33,"""33:33""",0.0,0.0,"""3M""",0.0,3.0,81.0,145.0,145.0
20241518,2024151801,1,2024,1,"""EYW""",33,"""FLL""",33,"""33:33""",0.0,0.0,"""3M""",0.0,6.0,89.0,145.0,145.0


In [32]:
df.dtypes

[Int64,
 Int64,
 Int64,
 Int64,
 Int64,
 String,
 Int64,
 String,
 Int64,
 String,
 Float64,
 Float64,
 String,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64]

In [33]:
rows = len(df)
print('Number of rows: ', rows)

Number of rows:  32766855


In [34]:
print(df.null_count())

shape: (1, 18)
┌─────────┬────────┬────────────────┬──────┬───┬────────────┬─────────────┬─────────────────┬────────────────────┐
│ ITIN_ID ┆ MKT_ID ┆ MARKET_COUPONS ┆ YEAR ┆ … ┆ PASSENGERS ┆ MARKET_FARE ┆ MARKET_DISTANCE ┆ MARKET_MILES_FLOWN │
│ ---     ┆ ---    ┆ ---            ┆ ---  ┆   ┆ ---        ┆ ---         ┆ ---             ┆ ---                │
│ u32     ┆ u32    ┆ u32            ┆ u32  ┆   ┆ u32        ┆ u32         ┆ u32             ┆ u32                │
╞═════════╪════════╪════════════════╪══════╪═══╪════════════╪═════════════╪═════════════════╪════════════════════╡
│ 0       ┆ 0      ┆ 0              ┆ 0    ┆ … ┆ 0          ┆ 0           ┆ 0               ┆ 0                  │
└─────────┴────────┴────────────────┴──────┴───┴────────────┴─────────────┴─────────────────┴────────────────────┘
